In [1]:
%matplotlib inline

In [2]:
def blend(color, factor):
    return int(255 - (255 - color) * factor)

def rgb(red, green, blue, factor=1):
    """Return color as #rrggbb for the given color values."""
    return '#%02x%02x%02x' % (blend(red, factor), blend(green, factor), blend(blue, factor))


colors = {
    'blue': rgb(55, 126, 184),
    'lightblue': rgb(55, 126, 184, 0.6),
    'oldgreen': rgb(77, 175, 74),
    'oldlightgreen': rgb(77, 175, 74, 0.6),
    'orange': rgb(255, 127, 0),
    'lightorange': rgb(255, 127, 0, 0.6),
    'red': rgb(228, 26, 28),
    'lightred': rgb(228, 26, 28, 0.75),
    'black': rgb(0, 0, 0),
    'morton':"#FFD200",
    'cuckoo':"#FF7F00",
    'lightmorton': rgb(255, 210, 0, 0.6),
    'lightcuckoo': rgb(255, 127, 0, 0.6),
    'xor':"#CD161C",
    'bloom':"#23507A",
    'lightbloom':rgb(35, 80, 122, 0.6),
    'fuse':"#29021d",
    'lightviolet':"#984EA3",
    'violet':"#67356F",
    'lightgreen': "#4DAF4A",
    'green': "#3C893A",
    'turquoise': "#45E2CD",
    'pink': "#F028F0",
}


from matplotlib.ticker import FuncFormatter


def kilos(x, pos):
    """The two args are the value and tick position"""
    return '%1.0f\\,K' % (x * 1e-3)


def millions(x, pos=None):
    """The two args are the value and tick position"""
    if x:
        return '%1.0f\\,M' % (x * 1e-6)
    else:
        return '0'


def billions(x, pos):
    """The two args are the value and tick position"""
    if x == 0:
        return '0'
    elif x < 1e8:
        return '%1.0f\\,M' % (x * 1e-6)
    elif x < 1e10:
        return '%1.1f\\,G' % (x * 1e-9)
    else:
        return '%1.0f\\,G' % (x * 1e-9)


def billions2(x, pos):
    """The two args are the value and tick position"""
    if x == 0:
        return '0'
    else:
        return '%1.0f\\,G' % (x * 1e-9)

def speedup(x, pos=None):
    sign = '+' if x > 0 else ''
    return sign + '%.0f' % (x * 100) + '\%'
    
def perc(x, pos):
    return '%.0f' % (x * 100) + '\%'


def perc2(x, pos):
    return '%.0f' % (x) + '\%'


kilos = FuncFormatter(kilos)
mills = FuncFormatter(millions)
gigs = FuncFormatter(billions)
gigs2 = FuncFormatter(billions2)
percent = FuncFormatter(perc)
percent2 = FuncFormatter(perc2)


markers = {
    'circle': 'o',
    'triangle': '^',
    'square': 's',
    'diamond': 'D'
}

def extract(rows: [dict], index: int, xaxis: str, yaxis: str):
    return rows[index][yaxis]


def extract_time(rows: [dict], index: int, xaxis: str, yaxis: str):
    return rows[index]['real_time']


def extract_throughput(rows: [dict], index: int, xaxis: str, yaxis: str):
    if rows[index]['fixture'] == 'Count':
        return rows[index]['n_elements_lookup'] * 1000 / rows[index]['real_time']
    else:
        return rows[index]['n_elements_build'] * 1000 / rows[index]['real_time']


def extract_speedup(rows: [dict], index: int, xaxis: str, yaxis: str):
    return rows[0]['real_time'] / rows[index]['real_time']


yconverter = {
    'time': extract_time,
    'throughput': extract_throughput,
    'speedup': extract_speedup,
    'DTLB-misses': extract,
    'ITLB-misses': extract,
    'L1D-misses': extract,
    'L1I-misses': extract,
    'LLC-misses': extract,
    'branch-misses': extract,
    'cycles': extract,
    'instructions': extract,
    'task-clock': extract,
    'avg_size': extract,
    'size': extract,
    'bits': extract,
    'retries': extract,
    'fpr': extract
}

xscale = {
    'k': 'linear',
    's': 'linear',
    'n_threads': 'linear',
    'n_partitions': 'log',
    'n_elements_build': 'log',
    'n_elements_lookup': 'log',
    'shared_elements': 'linear'
}


import pandas as pd


def read_benchmark(path: str):
    csv = pd.read_csv(path)

    split_name = csv['name'].apply(lambda x: x.split('/')[0].split('_'))

    csv['k'] = split_name.apply(lambda x: int(x[len(x) - 1]))
    csv['fixture'] = csv['name'].apply(lambda x: x.split('/')[1])
    csv['s'] = csv['name'].apply(lambda x: float(x.split('/')[2]) / 100)
    csv['n_threads'] = csv['name'].apply(lambda x: int(x.split('/')[3]))
    csv['n_partitions'] = csv['name'].apply(lambda x: int(x.split('/')[4]))
    csv['n_elements_build'] = csv['name'].apply(lambda x: int(x.split('/')[5]))
    csv['n_elements_lookup'] = csv['name'].apply(lambda x: int(x.split('/')[6]))
    csv['shared_elements'] = csv['name'].apply(lambda x: float(x.split('/')[7]) / 100)

    csv['name'] = split_name.apply(lambda x: "_".join(x[0:(len(x) - 1)]))

    data = {}
    for _, row in csv.iterrows():
        data_row = {}
        for label, item in row.iteritems():
            data_row[label] = item

        if data_row['fixture'] == 'Construct':
            data_row['throughput'] = data_row['n_elements_build'] / (data_row['duration'] / 1000)
        elif data_row['fixture'] == 'Count' or data_row['fixture'] == 'MTCount':
            data_row['throughput'] = data_row['n_elements_lookup'] / (data_row['duration'] / 1000)
            
        name = row['name']
        if name not in data:
            data[name] = []
        data[name].append(data_row)

    return data


import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, FuncFormatter, FixedLocator
from matplotlib.patches import Rectangle
import tikzplotlib
import numpy as np

matplotlib.use('pgf')

def latexify(fig_width=None, fig_height=None, columns=1):
    """Set up matplotlib's RC params for LaTeX plotting.
    Call this before plotting a figure.

    Parameters
    ----------
    fig_width : float, optional, inches
    fig_height : float,  optional, inches
    columns : {1, 2}
    """

    # code adapted from http://www.scipy.org/Cookbook/Matplotlib/LaTeX_Examples

    # Width and max height in inches for IEEE journals taken from
    # computer.org/cms/Computer.org/Journal%20templates/transactions_art_guide.pdf

    assert (columns in [1, 2])

    if fig_width is None:
        fig_width = 3.39 if columns == 1 else 6.9  # width in inches

    if fig_height is None:
        golden_mean = (np.sqrt(5) - 1.0) / 2.0  # Aesthetic ratio
        fig_height = fig_width * golden_mean  # height in inches

    MAX_HEIGHT_INCHES = 32.0
    if fig_height > MAX_HEIGHT_INCHES:
        print("WARNING: fig_height too large:" + fig_height +
              "so will reduce to" + MAX_HEIGHT_INCHES + "inches.")
        fig_height = MAX_HEIGHT_INCHES

    params = {'backend': 'ps',
              'pgf.rcfonts': False,
              'axes.labelsize': 7,  # fontsize for x and y labels (was 10)
              'axes.titlesize': 7,
              'font.size': 7,  # was 10
              'legend.fontsize': 7,  # was 8 # was 10
              'legend.handlelength': 1,
              'legend.handletextpad': 0.5,
              'legend.labelspacing': 0.1, # was 0.1
              'legend.columnspacing': 1.5,
              'legend.borderpad': 0.3,
              'xtick.labelsize': 7,
              'ytick.labelsize': 7,
              'axes.labelpad': 1,
              'axes.titlepad': 3,
              'text.usetex': True,
              'figure.figsize': [fig_width, fig_height],
              'font.family': 'serif',
              'text.latex.preamble': r'\usepackage{hyperref} \usepackage{amssymb} \usepackage{ifsym} \usepackage[T1]{fontenc} \usepackage{libertine} \usepackage{graphicx}',
              'pgf.preamble': r'\usepackage{hyperref} \usepackage{amssymb} \usepackage{ifsym} \usepackage[T1]{fontenc} \usepackage{libertine} \usepackage{graphicx}'
              }
    
    matplotlib.rcParams.update(params)

def logPrintFormat(x, pos):
    if x < 1:
        return "$10^{{\\scalebox{{0.75}}[1.0]{{-}}{}}}$".format(round(-math.log10(x)))
    else:
        return "$10^{}$".format(round(math.log10(x)))

def format_axes(ax, xscale='linear', yscale='linear'):
    spine_color = 'black'
    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)

    for spine in ['left', 'bottom']:
        ax.spines[spine].set_color(spine_color)
        ax.spines[spine].set_linewidth(0.5)

    ax.set_xscale(xscale)
    ax.set_yscale(yscale)
    
    if yscale == 'log':  
        locmaj = matplotlib.ticker.LogLocator(base=10,numticks=12) 
        ax.yaxis.set_major_locator(locmaj)
        ax.yaxis.set_major_formatter(FuncFormatter(logPrintFormat))
        locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(np.arange(0, 1, 0.1)),numticks=12)
        ax.yaxis.set_minor_locator(locmin)
        ax.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
    else:
        ax.yaxis.set_minor_locator(AutoMinorLocator(n=2))
        ax.yaxis.grid(b=True, which='minor', linestyle=':')

        
    ax.xaxis.set_ticks_position('bottom')
    ax.xaxis.set_tick_params(direction='out', color=spine_color)
    
    if xscale == 'log':  
        locmaj = matplotlib.ticker.LogLocator(base=10,numticks=12) 
        ax.xaxis.set_major_locator(locmaj)
        ax.xaxis.set_major_formatter(FuncFormatter(logPrintFormat))
        locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(np.arange(0, 1, 0.1)),numticks=12)
        ax.xaxis.set_minor_locator(locmin)
        ax.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
    
    ax.yaxis.set_ticks_position('left')
    ax.yaxis.set_tick_params(direction='out', color=spine_color)
    ax.yaxis.grid(b=True, which='major')

    ax.tick_params(axis='both', which='major', pad=0.5)

    return ax

def format_axins(ax, xscale='linear', yscale='linear'):
    spine_color = 'black'
    for spine in ['left', 'top', 'right', 'bottom']:
        ax.spines[spine].set_color(spine_color)
        ax.spines[spine].set_linewidth(0.5)

    ax.set_xscale(xscale)
    ax.set_yscale(yscale)
    
    ax.xaxis.set_visible(False)
    #ax.yaxis.set_visible(False)
    
    if yscale == 'log':  
        locmaj = matplotlib.ticker.LogLocator(base=10,numticks=12) 
        ax.yaxis.set_major_locator(locmaj)
        ax.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
    else:
        ax.yaxis.grid(b=True, which='minor', linestyle=':')

    ax.yaxis.grid(b=True, which='major')
    ax.set_yticklabels([])
    ax.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())

    for tic in ax.yaxis.get_major_ticks():
        tic.tick1line.set_visible(False)   
    for tic in ax.yaxis.get_minor_ticks():
        tic.tick1line.set_visible(False)  
        
    return ax


def barAxes(ax):
    ax.set_axisbelow(True)


def cm2inch(value):
    return value / 2.54


def reorderLegend(ax=None, order=None, unique=False):
    if ax is None: ax = plt.gca()
    handles, labels = ax.get_legend_handles_labels()
    labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))  # sort both labels and handles by labels
    if order is not None:  # Sort according to a given list (not necessarily complete)
        keys = dict(zip(order, range(len(order))))
        labels, handles = zip(*sorted(zip(labels, handles), key=lambda t, keys=keys: keys.get(t[0], np.inf)))
    if unique:
        labels, handles = zip(*unique_everseen(zip(labels, handles), key=labels))  # Keep only the first of each handle
    return handles, labels


def unique_everseen(seq, key=None):
    seen = set()
    seen_add = seen.add
    return [x for x, k in zip(seq, key) if not (k in seen or seen_add(k))]

import os

def savefig(path):
    dir = os.path.dirname(path)
    if not os.path.exists(dir):
        os.makedirs(dir)
    plt.savefig(path + ".pdf", bbox_inches='tight', pad_inches = 0)
    plt.savefig(path + ".pgf", bbox_inches='tight', pad_inches = 0)

In [3]:
import math

def analyzeFPR(data, skyline):
    for name in data.keys():
        b = 0
        cr = 0
        ota = 0
        
        if 'Cuckoo' in name and not 'Opt' in name:
            b = int(name.replace('Cuckoo', ''))
            skyline_name = 'Cuckoo'
        elif 'Morton' in name and not 'Opt' in name:
            b = int(name.replace('Morton', '').split('_')[0])
            cr = int(name.replace('Morton', '').split('_')[1])
            ota = int(name.replace('Morton', '').split('_')[2])
            skyline_name = f'Morton'
        else:
            b = 0
            skyline_name = name
            
        if skyline_name not in skyline.keys():
            skyline[skyline_name] = {}

        last_bits = 0
        
        for benchmark in data[name]:
            k = benchmark['k']
            bits = round(benchmark['bits']*4)/4
            failures = benchmark['failures']
            fpr = benchmark['fpr']
            s = benchmark['s']
            
            if failures == 100:
                bits = round(k*s*4)/4
            if bits == last_bits:
                bits += 0.25
            last_bits = bits

            if failures <= 0 and (bits not in skyline[skyline_name].keys() or skyline[skyline_name][bits]['fpr'] > fpr):
                skyline[skyline_name][bits] = {'k': k, 'fpr': fpr, 's': s, 'b': b, 'cr': cr, 'ota': ota}

In [4]:
def plotFPR(config, skyline, ax, yaxis):
    ax.set_xlabel('Bits per key ($m/n$)')
    ax.set_xlim(4.75, 25.25)
    handles = []
    for name in config.keys():
        x = []
        y = []
        for bits in sorted(list(skyline[name])):
            if 5 <= bits <= 25:
                x.append(bits)
                y.append(skyline[name][bits][yaxis])
        handles.append(ax.plot(x, y, label=config[name]['label'], color=config[name]['color'], linestyle=config[name]['linestyle'], linewidth=config[name]['linewidth'])[0])
    return handles

In [14]:
skyline = {}

analyzeFPR(read_benchmark('../benchmark/paper/background/fingerprintbased/vqf/fingerprintbased_vqf_fpr.csv'), skyline)
analyzeFPR(read_benchmark('../benchmark/paper/background/fingerprintbased/cuckoo/fingerprintbased_cuckoo_fpr.csv'), skyline)
analyzeFPR(read_benchmark('../benchmark/paper/background/fingerprintbased/cuckoo_opt/fingerprintbased_cuckoo_opt_fpr.csv'), skyline)
analyzeFPR(read_benchmark('../benchmark/paper/background/fingerprintbased/morton_opt/fingerprintbased_morton_opt_fpr.csv'), skyline)
analyzeFPR(read_benchmark('../benchmark/paper/background/fingerprintbased/morton2/fingerprintbased_morton2_fpr.csv'), skyline)
analyzeFPR(read_benchmark('../benchmark/paper/background/fingerprintbased/morton3/fingerprintbased_morton3_fpr.csv'), skyline)
analyzeFPR(read_benchmark('../benchmark/paper/background/fingerprintbased/morton7/fingerprintbased_morton7_fpr.csv'), skyline)
analyzeFPR(read_benchmark('../benchmark/paper/background/fingerprintbased/xor/fingerprintbased_xor_fpr.csv'), skyline)
analyzeFPR(read_benchmark('../benchmark/paper/background/fingerprintbased/fuse/fingerprintbased_fuse_fpr.csv'), skyline)
analyzeFPR(read_benchmark('../benchmark/paper/background/bloom/bloom_fpr.csv'), skyline)
                
configFPR = {
    'VQF': {'label': 'VQF', 'color': colors['blue'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 1},
    'BloomStandard': {'label': 'Bloom', 'color': colors['pink'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 1},
    'Cuckoo': {'label': '', 'color': colors['lightgreen'], 'marker': markers['circle'], 'linestyle': 'dashed', 'linewidth': 1},
    'CuckooOpt': {'label': 'Cuckoo', 'color': colors['green'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 1},
    'Morton': {'label': '', 'color': colors['lightorange'], 'marker': markers['circle'], 'linestyle': 'dashed', 'linewidth': 1},
    'MortonOpt': {'label': 'Morton', 'color': colors['orange'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 1},
    'Xor': {'label': 'Xor', 'color': colors['red'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 1},
    'Fuse': {'label': 'Xor (Fuse)', 'color': colors['lightred'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 1},
}

configFPR_fingerprint = {
    'VQF': {'label': 'VQF', 'color': colors['blue'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 1},
    #'BloomStandard': {'label': 'Bloom', 'color': colors['bloom'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 1},
    'Cuckoo': {'label': '', 'color': colors['lightgreen'], 'marker': markers['circle'], 'linestyle': 'dashed', 'linewidth': 1},
    'CuckooOpt': {'label': 'Cuckoo', 'color': colors['green'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 1},
    'Morton': {'label': '', 'color': colors['lightorange'], 'marker': markers['circle'], 'linestyle': 'dashed', 'linewidth': 1},
    'MortonOpt': {'label': 'Morton', 'color': colors['orange'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 1},
    'Xor': {'label': 'Xor', 'color': colors['red'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 1},
    'Fuse': {'label': 'Xor (Fuse)', 'color': colors['lightred'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 1},
}

configK = {
    'VQF': {'label': 'VQF', 'color': colors['blue'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 1},
    'CuckooOpt': {'label': 'Cuckoo', 'color': colors['green'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 1},
    'MortonOpt': {'label': 'Morton', 'color': colors['orange'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 1},
    'Xor': {'label': 'Xor', 'color': colors['red'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 1},
    'Fuse': {'label': 'Xor (Fuse)', 'color': colors['lightred'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 1},
       }

def plot(config, ax, yaxis):
    for name in config.keys():
        x = []
        y = []
        for bits in sorted(list(skyline[name])):
            if 5 <= bits <= 25:
                x.append(bits)
                y.append(skyline[name][bits][yaxis])
        ax.plot(x, y, label=config[name]['label'], color=config[name]['color'], linestyle=config[name]['linestyle'], linewidth=config[name]['linewidth'])

latexify(cm2inch(8.5), cm2inch(5),2)

fig = plt.figure()
ax = fig.add_subplot(111)
plot(configK, ax, 'k')
ax.legend().set_visible(True)
ax.set_ylabel("optimal $r$")
ax.set_xlabel("bits per key ($m/n$)")
format_axes(ax)
plt.savefig("./pdf/background/fingerprintbased/fingerprintbased_k.pdf", bbox_inches='tight', pad_inches = 0)

fig = plt.figure()
ax = fig.add_subplot(111)
plot(configFPR_fingerprint, ax, 's')
ax.legend().set_visible(True)
ax.set_ylabel("optimal $s$")
ax.set_xlabel("bits per key ($m/n$)")
format_axes(ax)
plt.savefig("./pdf/background/fingerprintbased/fingerprintbased_s.pdf", bbox_inches='tight', pad_inches = 0)

fig = plt.figure()
ax = fig.add_subplot(111)
plot(configFPR, ax, 'fpr')
ax.legend().set_visible(True)
ax.set_ylabel("false-positive rate $\\varepsilon$")
ax.set_xlabel("bits per key ($m/n$)")
format_axes(ax, 'linear', 'log')
plt.savefig("./pdf/background/fingerprintbased/vqf_fpr_comp.pdf", bbox_inches='tight', pad_inches = 0)

/tmp/ipykernel_988/1209303967.py:56: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()
/tmp/ipykernel_988/1300565947.py:277: MatplotlibDeprecationWarning: The 'b' parameter of grid() has been renamed 'visible' since Matplotlib 3.5; support for the old name will be dropped two minor releases later.
  ax.yaxis.grid(b=True, which='minor', linestyle=':')
/tmp/ipykernel_988/1300565947.py:293: MatplotlibDeprecationWarning: The 'b' parameter of grid() has been renamed 'visible' since Matplotlib 3.5; support for the old name will be dropped two minor releases later.
  ax.yaxis.grid(b=True, which='major')
/tmp/ipykernel_988/1300565947.py:277: MatplotlibDeprecationWarning: The 'b' parameter of grid() has been renamed 'visible' since Matplotlib 3.5; support for

In [12]:
import math


skyline = {}

def analyze(data):
    for name in data.keys():
        skyline[name] = {}
        last_bits = 0
        for benchmark in data[name]:
            k = benchmark['k']
            failures = benchmark['failures']
            fpr = benchmark['fpr']
            s = benchmark['s']
            if failures == 0 and (k not in skyline[name].keys() or skyline[name][k]['s'] > s):
                skyline[name][k] = {'k': k, 'fpr': fpr, 's': s}

analyze(read_benchmark('../benchmark/paper/background/fingerprintbased/vqf/fingerprintbased_vqf_failure_k.csv')) 
analyze(read_benchmark('../benchmark/paper/background/fingerprintbased/xor/fingerprintbased_xor_failure_k.csv')) 
analyze(read_benchmark('../benchmark/paper/background/fingerprintbased/morton_opt/fingerprintbased_morton_opt_failure_k.csv')) 
analyze(read_benchmark('../benchmark/paper/background/fingerprintbased/cuckoo_opt/fingerprintbased_cuckoo_opt_failure_k.csv')) 
analyze(read_benchmark('../benchmark/paper/background/fingerprintbased/fuse/fingerprintbased_fuse_failure_k.csv')) 

config = {
    'VQF': {'label': 'VQF', 'color': colors['blue'], 'marker': markers['circle']},
    'CuckooOpt': {'label': 'Cuckoo', 'color': colors['green'], 'marker': markers['circle']},
    'MortonOpt': {'label': 'Morton', 'color': colors['orange'], 'marker': markers['circle']},
    'Xor': {'label': 'Xor', 'color': colors['red'], 'marker': markers['circle']},
    'Fuse': {'label': 'Xor (Fuse)', 'color': colors['lightred'], 'marker': markers['circle']},
         }

def plot(ax, yaxis):
    for name in config.keys():
        x = []
        y = []
        for k in sorted(list(skyline[name])):
            if 1 <= k <= 32:
                x.append(k)
                y.append(skyline[name][k][yaxis])
        ax.plot(x, y, label=config[name]['label'], color=config[name]['color'], linewidth=1)

latexify(cm2inch(8.5), cm2inch(5),2)

fig = plt.figure()
ax = fig.add_subplot(111)
plot(ax, 's')
ax.legend().set_visible(True)
ax.set_ylabel("minimal $s$")
ax.set_xlabel("fingerprint size $r$ [bits]")
format_axes(ax)
plt.savefig("./pdf/background/fingerprintbased/fingerprintbased_failure_k_s.pdf", bbox_inches='tight', pad_inches = 0)

/tmp/ipykernel_988/1300565947.py:277: MatplotlibDeprecationWarning: The 'b' parameter of grid() has been renamed 'visible' since Matplotlib 3.5; support for the old name will be dropped two minor releases later.
  ax.yaxis.grid(b=True, which='minor', linestyle=':')
/tmp/ipykernel_988/1300565947.py:293: MatplotlibDeprecationWarning: The 'b' parameter of grid() has been renamed 'visible' since Matplotlib 3.5; support for the old name will be dropped two minor releases later.
  ax.yaxis.grid(b=True, which='major')


In [6]:
skyline = {}

def analyze(data):
    for name in data.keys():
        skyline[name] = {}
        last_bits = 0
        for benchmark in data[name]:
            elements = benchmark['n_elements_build']
            failures = benchmark['failures']
            fpr = benchmark['fpr']
            s = benchmark['s']
            if benchmark['bits'] <= s*16*1.05 and failures == 0 and (elements not in skyline[name].keys() or skyline[name][elements]['s'] > s):
                skyline[name][elements] = {'elements': elements, 'fpr': fpr, 's': s}

analyze(read_benchmark('../benchmark/paper/background/fingerprintbased/vqf/fingerprintbased_vqf_failure_elements.csv'))
analyze(read_benchmark('../benchmark/paper/background/fingerprintbased/xor/fingerprintbased_xor_failure_elements.csv')) 
analyze(read_benchmark('../benchmark/paper/background/fingerprintbased/fuse/fingerprintbased_fuse_failure_elements.csv')) 
analyze(read_benchmark('../benchmark/paper/background/fingerprintbased/morton_opt/fingerprintbased_morton_opt_failure_elements.csv')) 
analyze(read_benchmark('../benchmark/paper/background/fingerprintbased/cuckoo_opt/fingerprintbased_cuckoo_opt_failure_elements.csv'))

config = {
    'VQF': {'label': 'VQF', 'color': colors['blue'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 1, 'min': 100},
    'CuckooOpt': {'label': 'Cuckoo', 'color': colors['violet'], 'marker': markers['circle'], 'min': 100},
    'MortonOpt': {'label': 'Morton', 'color': colors['green'], 'marker': markers['circle'], 'min': 1000},
    'Xor': {'label': 'Xor', 'color': colors['red'], 'marker': markers['circle'], 'min': 100},
    'Fuse': {'label': 'Xor (Fuse)', 'color': colors['lightred'], 'marker': markers['circle'], 'min': 100},
}

def plot(ax, yaxis):
    for name in config.keys():
        x = []
        y = []
        for elements in sorted(list(skyline[name])):
            if elements >= config[name]['min']:
                x.append(elements)
                y.append(skyline[name][elements][yaxis])
        ax.plot(x, y, label=config[name]['label'], color=config[name]['color'], linewidth=1)

latexify(cm2inch(8.5), cm2inch(5),2)

fig = plt.figure()
ax = fig.add_subplot(111)
plot(ax, 's')
ax.legend().set_visible(True)
ax.set_ylabel("optimal $s$")
ax.set_xlabel("\#elements")
format_axes(ax, "log", "linear")
plt.savefig("./pdf/background/fingerprintbased/fingerprintbased_vqf_failure_elements_s.pdf", bbox_inches='tight', pad_inches = 0)

/tmp/ipykernel_461/1300565947.py:277: MatplotlibDeprecationWarning: The 'b' parameter of grid() has been renamed 'visible' since Matplotlib 3.5; support for the old name will be dropped two minor releases later.
  ax.yaxis.grid(b=True, which='minor', linestyle=':')
/tmp/ipykernel_461/1300565947.py:293: MatplotlibDeprecationWarning: The 'b' parameter of grid() has been renamed 'visible' since Matplotlib 3.5; support for the old name will be dropped two minor releases later.
  ax.yaxis.grid(b=True, which='major')


In [5]:
skyline = {}

analyzeFPR(read_benchmark('../benchmark/paper/background/fingerprintbased/vqf/fingerprintbased_vqf_hasher_magic.csv'), skyline)
                
configFPR_zoomed_out = {
    'VQF_Murmur': {'label': 'Murmur', 'color': colors['blue'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 0.5},
    'VQF_Fasthash': {'label': 'FastHash', 'color': colors['green'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 0.4},
    'VQF_Cityhash': {'label': 'CityHash', 'color': colors['red'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 0.4},
    'VQF_TwoIndependentMultiplyShift': {'label': 'TwoIndependentMultiplyShift', 'color': colors['orange'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 0.4},
    'VQF_Mul': {'label': 'Mul', 'color': colors['black'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 0.4},
}

configFPR = {
    'VQF_Murmur': {'label': 'Murmur', 'color': colors['blue'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 0.5},
    'VQF_Fasthash': {'label': 'FastHash', 'color': colors['green'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 0.4},
    'VQF_Cityhash': {'label': 'CityHash', 'color': colors['red'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 0.4},
    'VQF_TwoIndependentMultiplyShift': {'label': 'TwoIndependentMultiplyShift', 'color': colors['orange'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 0.4},
}

def plot(config, ax, yaxis):
    for name in config.keys():
        x = []
        y = []
        for bits in sorted(list(skyline[name])):
            if 17 <= bits <= 25:
                x.append(bits)
                y.append(skyline[name][bits][yaxis])
        ax.plot(x, y, label=config[name]['label'], color=config[name]['color'], linestyle=config[name]['linestyle'], linewidth=config[name]['linewidth'])

latexify(cm2inch(8.5), cm2inch(5),2)

fig = plt.figure()
ax = fig.add_subplot(111)
plot(configFPR, ax, 'fpr')
ax.legend().set_visible(True)
ax.set_ylabel("false-positive rate $\\varepsilon$")
ax.set_xlabel("bits per key ($m/n$)")
format_axes(ax, 'linear', 'linear')
plt.savefig("./pdf/background/fingerprintbased/vqf_hasher.pdf", bbox_inches='tight', pad_inches = 0.1)

def plot(config, ax, yaxis):
    for name in config.keys():
        x = []
        y = []
        for bits in sorted(list(skyline[name])):
            if 5 <= bits <= 25:
                x.append(bits)
                y.append(skyline[name][bits][yaxis])
        ax.plot(x, y, label=config[name]['label'], color=config[name]['color'], linestyle=config[name]['linestyle'], linewidth=config[name]['linewidth'])

latexify(cm2inch(8.5), cm2inch(5),2)
        
fig = plt.figure()
ax = fig.add_subplot(111)
plot(configFPR_zoomed_out, ax, 'fpr')
ax.legend().set_visible(True)
ax.set_ylabel("false-positive rate $\\varepsilon$")
ax.set_xlabel("bits per key ($m/n$)")
format_axes(ax, 'linear', 'log')
plt.savefig("./pdf/background/fingerprintbased/vqf_hasher_zoomed_out.pdf", bbox_inches='tight', pad_inches = 0)

/tmp/ipykernel_1119/1300565947.py:277: MatplotlibDeprecationWarning: The 'b' parameter of grid() has been renamed 'visible' since Matplotlib 3.5; support for the old name will be dropped two minor releases later.
  ax.yaxis.grid(b=True, which='minor', linestyle=':')
/tmp/ipykernel_1119/1300565947.py:293: MatplotlibDeprecationWarning: The 'b' parameter of grid() has been renamed 'visible' since Matplotlib 3.5; support for the old name will be dropped two minor releases later.
  ax.yaxis.grid(b=True, which='major')
/tmp/ipykernel_1119/1300565947.py:293: MatplotlibDeprecationWarning: The 'b' parameter of grid() has been renamed 'visible' since Matplotlib 3.5; support for the old name will be dropped two minor releases later.
  ax.yaxis.grid(b=True, which='major')


In [19]:
skyline = {}

analyzeFPR(read_benchmark('../benchmark/paper/background/fingerprintbased/vqf/fingerprintbased_vqf_fpr.csv'), skyline)
analyzeFPR(read_benchmark('../benchmark/paper/optimization/simd/vqf_fpr_avx512.csv'), skyline)
                
configFPR = {
    'VQF': {'label': 'Scalar', 'color': colors['blue'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 1},
    'VQF_AVX512': {'label': 'AVX512', 'color': colors['green'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 1},
}

def plot(config, ax, yaxis):
    for name in config.keys():
        x = []
        y = []
        for bits in sorted(list(skyline[name])):
            if 17 <= bits <= 25:
                x.append(bits)
                y.append(skyline[name][bits][yaxis])
        ax.plot(x, y, label=config[name]['label'], color=config[name]['color'], linestyle=config[name]['linestyle'], linewidth=config[name]['linewidth'])

latexify(cm2inch(8.5), cm2inch(5),2)

fig = plt.figure()
ax = fig.add_subplot(111)
plot(configFPR, ax, 'fpr')
ax.legend().set_visible(True)
ax.set_ylabel("false-positive rate $\\varepsilon$")
ax.set_xlabel("bits per key ($m/n$)")
format_axes(ax, 'linear', 'log')
plt.savefig("./pdf/background/fingerprintbased/vqf_fpr_avx512_comp.pdf", bbox_inches='tight', pad_inches = 0)

/tmp/ipykernel_700/1300565947.py:293: MatplotlibDeprecationWarning: The 'b' parameter of grid() has been renamed 'visible' since Matplotlib 3.5; support for the old name will be dropped two minor releases later.
  ax.yaxis.grid(b=True, which='major')


In [15]:
skyline = {}

analyzeFPR(read_benchmark('../benchmark/paper/background/fingerprintbased/vqf/fingerprintbased_vqf_addresser.csv'), skyline)
                
configFPR = {
    'VQF_Lemire': {'label': 'Ross', 'color': colors['blue'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 1},
    'VQF_PowerOfTwo': {'label': 'PowerOfTwo', 'color': colors['green'], 'marker': markers['circle'], 'linestyle': 'solid', 'linewidth': 1},
}
def plot(config, ax, yaxis):
    for name in config.keys():
        x = []
        y = []
        for bits in sorted(list(skyline[name])):
            if 5 <= bits <= 30:
                x.append(bits)
                y.append(skyline[name][bits][yaxis])
        ax.plot(x, y, label=config[name]['label'], color=config[name]['color'], linestyle=config[name]['linestyle'], linewidth=config[name]['linewidth'])

latexify(cm2inch(8.5), cm2inch(5),2)
        
fig = plt.figure()
ax = fig.add_subplot(111)
plot(configFPR, ax, 'fpr')
ax.legend().set_visible(True)
ax.set_ylabel("false-positive rate $\\varepsilon$")
ax.set_xlabel("bits per key ($m/n$)")
format_axes(ax, 'linear', 'log')
plt.savefig("./pdf/background/fingerprintbased/vqf_addresser.pdf", bbox_inches='tight', pad_inches = 0)

/tmp/ipykernel_700/1300565947.py:293: MatplotlibDeprecationWarning: The 'b' parameter of grid() has been renamed 'visible' since Matplotlib 3.5; support for the old name will be dropped two minor releases later.
  ax.yaxis.grid(b=True, which='major')
